### Imports

In [1]:
from transformers import pipeline,BartTokenizer,AutoTokenizer,AutoModelForQuestionAnswering,AutoModelForSeq2SeqLM,AutoModelForSequenceClassification,AutoModel
import requests,re,nltk,torch,numpy as np
from bs4 import BeautifulSoup
import urllib.parse
from sklearn.metrics.pairwise import cosine_similarity
from os.path import exists
from random import sample as random_sample
from spacy import load as spc_load

/home/user/mlenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-19 02:17:34.005270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739911654.025575   21980 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739911654.031519   21980 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 02:17:34.053137: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.

### Session context

In [2]:
class SessionManager:
    def __init__(self, max_tokens=1024):
        self.tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
        self.max_tokens = max_tokens
        self.context = ""

    def update_context(self, new_content):
        combined = f"{self.context}\n{new_content}".strip()
        tokens = self.tokenizer.tokenize(combined)
        if len(tokens) > self.max_tokens:
            keep_tokens = int(self.max_tokens * 0.35)
            new_tokens = self.tokenizer.tokenize(new_content)
            tokens = tokens[:keep_tokens] + new_tokens[-(self.max_tokens - keep_tokens):]
            
        self.context = self.tokenizer.convert_tokens_to_string(tokens)
    
    def get_context(self): return self.context.strip()
session = SessionManager()

### Preprocessing

In [3]:
def extract_text(html):
    text = re.sub(r'<[^>]*>',' ',html)
    text = re.sub(r'\s+',' ',text)
    text = re.sub(r'[?:;\'"(){}[\]<>|@&\/\\]','',text)
    text = re.sub(r'%[^%]*%','',text)
    text = re.sub(r'%','',text)
    words = text.split()
    filtered_words = [word for word in words if not (len(word) > 45 or (word.isupper() and len(word) > 32))]
    text = " ".join(filtered_words)
    return text.strip()
session = SessionManager()
user_query = input("Enter your query:").lower()
print(user_query)
full_query = f"{session.get_context()}\n\nCurrent Query: {user_query}"
search_url = f"https://html.duckduckgo.com/html/?q={'+'.join(user_query.split(' '))}"
response = requests.get(search_url,headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:135.0) Gecko/20100101 Firefox/135.0"
})

who discovered the electron


In [4]:
def res_urls(response):
    soup = BeautifulSoup(response.text,"html.parser")
    sresults = []
    for i,result in enumerate(soup.find_all("div",class_="result"),start=1):
        if i > 6: break
        title = result.find("a",class_="result__a")
        if not title:continue
        tag = result.find("a",class_="result__snippet")
        snip = tag.text.strip() if tag else None
        sresults.append({
            "id":i,
            "url":title["href"],
            "description":snip
        })
    return sresults
search_results = res_urls(response)
content_dict = {}
for result in search_results:
    try:
        url = result["url"]
        mch = re.search(r"uddg=([^&]+)",url)
        if mch:url = urllib.parse.unquote(mch.group(1))
        page_response = requests.get(url,headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:135.0) Gecko/20100101 Firefox/135.0"})
        soup = BeautifulSoup(page_response.text,"html.parser")
        for element in soup(["script","style","nav","footer","header"]):element.decompose()
        page_text = extract_text(page_response.text)[:7500]
        if page_text:
            content_dict[result['id']] = page_text
    except Exception:continue

### Relevance scoring

In [5]:
classifier = pipeline("zero-shot-classification",model="cross-encoder/nli-roberta-base")
def filter_relevant_sentences(text, query, threshold=0.7):
    sentences = nltk.sent_tokenize(text)
    candidate_labels = ["entailment", "contradiction"]
    hypothesis_template = f"This sentence {{}} is relevant to the query: {query}"
    valid_sentences = [s for s in sentences if s.strip()]
    if not valid_sentences:
        return ""
    
    results = classifier(
        valid_sentences, 
        candidate_labels,
        hypothesis_template=hypothesis_template,
        batch_size=4
    )
    
    filtered_sentences = []
    for sentence, result in zip(valid_sentences, results):
        score = result["scores"][result["labels"].index("entailment")]
        if score >= threshold:
            filtered_sentences.append(sentence)
    
    return " ".join(filtered_sentences)

Device set to use cuda:0


### Summarization

In [6]:
tokenizer_sum = BartTokenizer.from_pretrained("facebook/bart-base")
summarizer = pipeline("summarization",model="facebook/bart-base")

Device set to use cuda:0


In [7]:
def chunk_text(text,max_tokens=512):
    global tokenizer_sum
    sentences = nltk.sent_tokenize(text,language="english")
    full_text_token_count = len(tokenizer_sum.tokenize(text))
    if full_text_token_count <= max_tokens: return [text]
    chunks = []
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        sentence_tokens = tokenizer_sum.tokenize(sentence)
        sentence_token_count = len(sentence_tokens)
        if current_length + sentence_token_count <= max_tokens:
            current_chunk.append(sentence)
            current_length += sentence_token_count
        else:
            if current_chunk:
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_length = sentence_token_count
            else:
                chunks.append(sentence[:512])
                current_chunk = []
                current_length = 0
    if current_chunk:chunks.append(" ".join(current_chunk))
    return chunks
def summarize_large_text(text,query,max_chunk_tokens=512,threshold=0.7):
    filtered_text = filter_relevant_sentences(text,query,threshold)
    if not filtered_text.strip():
        return ""
    chunks = chunk_text(filtered_text,max_chunk_tokens)
    summaries = []
    for chunk in chunks:
        if not chunk.strip():continue
        try:
            input_tokens = tokenizer_sum.tokenize(chunk)
            input_length = len(input_tokens)
            adjusted_max_length = min(200,input_length) if input_length > 10 else 10
            adjusted_min_length = min(50,adjusted_max_length - 1) if adjusted_max_length > 1 else 0

            summary = summarizer(
                chunk,
                max_length=adjusted_max_length,
                min_length=adjusted_min_length,
                do_sample=False,
                truncation=True
            )[0]["summary_text"]
            summaries.append(summary)
        except Exception:continue
    if not summaries: return ""
    final_summary = " ".join(summaries)
    while len(tokenizer_sum.tokenize(final_summary)) > 512:
        input_tokens = tokenizer_sum.tokenize(final_summary)
        input_length = len(input_tokens)
        adjusted_max_length = min(512,input_length)
        adjusted_min_length = min(100,adjusted_max_length - 1) if adjusted_max_length > 1 else 0
        final_summary = summarizer(
            final_summary,
            max_length=adjusted_max_length,
            min_length=adjusted_min_length,
            do_sample=False,
            truncation=True
        )[0]["summary_text"]
    return final_summary

In [8]:
page_summaries = {}
for doc_id,text in content_dict.items():
    page_prompt = (
        f"Extract information relevant to '{user_query}' from this text:\n\n{text}"
    )
    page_summary = summarize_large_text(page_prompt,user_query)
    page_summaries[doc_id] = page_summary
combined_summaries = " ".join(page_summaries.values())
final_prompt = (
    f"Considering previous context: {session.get_context()}\n"
    f"Synthesize information relevant to '{user_query}' from these summaries:\n\n{combined_summaries}"
)
sumry = summarize_large_text(final_prompt,full_query)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Augmented Retrieval

In [9]:
tokenizer_emb = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model_emb = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2").eval()
tokenizer_rk = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
model_rk = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
def get_embedding(text):
    global tokenizer_emb,model_emb
    inputs = tokenizer_emb(text,return_tensors="pt",truncation=True,max_length=256)
    with torch.no_grad():outputs = model_emb(**inputs)
    embeddings = outputs.last_hidden_state
    attention_mask = inputs["attention_mask"]
    masked_embeddings = embeddings * attention_mask.unsqueeze(-1).float()
    mean_embedding = masked_embeddings.sum(dim=1) / attention_mask.sum(dim=1,keepdim=True).clamp(min=1e-9)
    return mean_embedding[0].numpy()
def rerank(query,contexts,top_k=4):
    global tokenizer_rk,model_rk
    scores = []
    for context in contexts:
        inputs = tokenizer_rk(query,context,return_tensors="pt",truncation=True)
        with torch.no_grad():
            scores.append(model_rk(**inputs).logits.item())
    return [c for _,c in sorted(zip(scores,contexts),reverse=True)[:top_k]]

In [10]:
rag_chunks = chunk_text(" ".join(content_dict.values()),256)
chunk_embeds = np.array([get_embedding(chunk) for chunk in rag_chunks])
query_embed = get_embedding(user_query).reshape(1,-1)
similarity_scores = cosine_similarity(query_embed,chunk_embeds)[0]
top_k = 4
ini_indices = np.argsort(similarity_scores)[:top_k]
ini_context = [rag_chunks[i] for i in ini_indices]
retrieved_context = " ".join(rerank(user_query,ini_context,top_k=4))

### Q&A

In [11]:
qa_pipeline = pipeline("question-answering",model="distilbert-base-cased-distilled-squad")
query_context = f"{sumry}\n\n{retrieved_context}"
qa_result = qa_pipeline(question=user_query,context=query_context)
print(qa_result["answer"])

Device set to use cuda:0


J.J. Thomson


In [12]:
session.update_context(
    f"query: {user_query}\nanswer: {qa_result['answer']}\n"
    f"summary context: {sumry}\nretrieved Context: {retrieved_context}"
)

### RLHF

In [13]:
class REINFORCETrainer:
    def __init__(self, model, tokenizer, learning_rate=4e-5):
        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
    def update(self,inputs,responses,rewards):
        rewards_tensor = torch.tensor(rewards, dtype=torch.float32).to(self.model.device)
        
        if isinstance(self.model, AutoModelForQuestionAnswering):
            outputs = self.model(**inputs)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits
            
            start_probs = torch.nn.functional.log_softmax(start_logits, dim=-1)
            end_probs = torch.nn.functional.log_softmax(end_logits, dim=-1)
            
            start_positions = responses["start_positions"]
            end_positions = responses["end_positions"]
            
            start_log_probs = start_probs.gather(-1, start_positions.unsqueeze(-1)).squeeze(-1)
            end_log_probs = end_probs.gather(-1, end_positions.unsqueeze(-1)).squeeze(-1)
            total_log_probs = (start_log_probs + end_log_probs) / 2

            loss = -torch.mean(total_log_probs * rewards_tensor)

        elif isinstance(self.model, AutoModelForSequenceClassification):
            outputs = self.model(**inputs)
            log_probs = torch.nn.functional.log_softmax(outputs.logits, dim=-1)
            response_labels = responses["labels"]
            selected_log_probs = log_probs.gather(-1, response_labels.unsqueeze(-1)).squeeze(-1)
            loss = -torch.mean(selected_log_probs * rewards_tensor)

        else:
            outputs = self.model(**inputs, labels=responses["input_ids"])
            log_probs = torch.nn.functional.log_softmax(outputs.logits, dim=-1)
            response_ids = responses["input_ids"]
            response_mask = (response_ids != self.tokenizer.pad_token_id).float()
            nll = torch.nn.functional.nll_loss(
                log_probs.view(-1, log_probs.size(-1)),
                response_ids.view(-1),
                reduction='none'
            )
            log_probs = -nll.view(response_ids.size()) * response_mask
            avg_log_probs = (log_probs.sum(dim=-1) / response_mask.sum(dim=-1)).mean()
            loss = -avg_log_probs * rewards_tensor.mean()
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [14]:
rl_factors = {
    "qa":0.5,
    "relevance":0.3,
    "summarizer":0.2
}

model_paths = {
    "summarizer":"../models/fine_tuned_bart",
    "qa":"../models/fine_tuned_qa",
    "relevance":"../models/fine_tuned_relevance"
}

def init_model(model_name,model_class):
    if exists(model_paths[model_name]):
        model = model_class.from_pretrained(model_paths[model_name])
        tokenizer = AutoTokenizer.from_pretrained(model_paths[model_name])
    else:
        model = model_class.from_pretrained({
            "summarizer":"facebook/bart-base",
            "qa":"distilbert-base-cased-distilled-squad",
            "relevance":"cross-encoder/nli-roberta-base"
        }[model_name])
        tokenizer = AutoTokenizer.from_pretrained({
            "summarizer":"facebook/bart-base",
            "qa":"distilbert-base-cased-distilled-squad",
            "relevance":"cross-encoder/nli-roberta-base"
        }[model_name])
    return model,tokenizer

In [15]:
model_summarizer,tokenizer_summarizer = init_model("summarizer",AutoModelForSeq2SeqLM)
model_qa,tokenizer_qa = init_model("qa",AutoModelForQuestionAnswering)
model_relevance,tokenizer_relevance = init_model("relevance",AutoModelForSequenceClassification)
relevance_inputs = tokenizer_relevance(text=[user_query],text_pair=[retrieved_context],padding=True,truncation=True,return_tensors="pt")
trainers = {
    "summarizer": REINFORCETrainer(model_summarizer, tokenizer_summarizer),
    "qa": REINFORCETrainer(model_qa, tokenizer_qa),
    "relevance": REINFORCETrainer(model_relevance, tokenizer_relevance)
}

In [16]:
def get_feedback():
    try:
        feedback = int(input("Rate the answer quality (1-5):"))
        return max(1,min(5,feedback))
    except ValueError:
        return 3
user_rating = get_feedback()
user_rating = max(1,min(5,user_rating))
base_reward = (user_rating - 3) / 2
rewards = {k:base_reward * v for k,v in rl_factors.items()}

In [17]:
negative_indices = np.argsort(similarity_scores)[:top_k]
positive_chunks = [rag_chunks[i] for i in ini_indices]
negative_chunks = [rag_chunks[i] for i in negative_indices]
context_pairs = [(user_query, chunk, 1) for chunk in positive_chunks] + \
                [(user_query, chunk, 0) for chunk in negative_chunks]
np.random.shuffle(context_pairs)

relevance_inputs = tokenizer_relevance.batch_encode_plus(
    [(session.get_context(), c) for _, c, _ in context_pairs],
    padding=True,
    truncation=True,
    return_tensors="pt"
)
relevance_labels = torch.tensor([l for _, _, l in context_pairs]).long()

with torch.no_grad():
    logits = model_relevance(**relevance_inputs).logits
    preds = torch.sigmoid(logits[:, 0]).round().squeeze()
    labels = torch.tensor([l for _, _, l in context_pairs]).float()
    accuracy = (preds == labels).float().mean()
rewards["relevance"] = accuracy.item() * rewards["relevance"]

In [18]:
all_qa_chunks = chunk_text(" ".join(content_dict.values()), 256)
query_context_chunks = chunk_text(query_context, 256)

negative_qa_chunks = [c for c in all_qa_chunks if c not in query_context_chunks]
adversarial_qa_chunks = random_sample(negative_qa_chunks, min(2, len(negative_qa_chunks)))

augmented_qa_context = f"{query_context}[ADV]{'[ADV]'.join(adversarial_qa_chunks)}"

answer_embed = get_embedding(qa_result["answer"])
context_embed = get_embedding(augmented_qa_context)
consistency_score = cosine_similarity([answer_embed], [context_embed])[0][0]
qa_reward = (
    0.5 * base_reward +
    0.3 * consistency_score +
    0.2 * cosine_similarity(
        [get_embedding(qa_result["answer"])],
        [get_embedding(session.get_context())]
    )[0][0]
)

In [19]:
nlp = spc_load("en_core_web_md")
doc_source = nlp(combined_summaries)
doc_summary = nlp(sumry)
source_ents = set((ent.text.lower(), ent.label_) for ent in doc_source.ents)
summary_ents = set((ent.text.lower(), ent.label_) for ent in doc_summary.ents)
entity_retention = len(summary_ents & source_ents) / len(source_ents) if source_ents else 1.0

compression_ratio = len(sumry) / len(combined_summaries)
compression_score = max(0, 1 - abs(0.45 - compression_ratio))

summarizer_reward = (
    0.3 * base_reward +
    0.4 * entity_retention +
    0.2 * compression_score +
    0.1 * cosine_similarity(
        [get_embedding(sumry)],
        [get_embedding(session.get_context())]
    )[0][0]
)

In [20]:
inputs = (
    {
        "name": "summarizer",
        "query": tokenizer_summarizer(final_prompt, return_tensors="pt", truncation=True),
        "response": tokenizer_summarizer(sumry, return_tensors="pt"),
        "reward": [summarizer_reward * rl_factors["summarizer"]]
    },
    {
        "name": "qa",
        "query": tokenizer_qa(
            user_query,
            augmented_qa_context,
            return_tensors="pt",
            truncation=True,
            padding=True
        ),
        "response": {
            "start_positions": torch.tensor([qa_result["start"]]),
            "end_positions": torch.tensor([qa_result["end"]])
        },
        "reward": [qa_reward * rl_factors["qa"]]
    },
    {
        "name": "relevance",
        "query": tokenizer_relevance.batch_encode_plus(
            [(session.get_context(), c) for _, c, _ in context_pairs],
            padding=True,
            truncation=True,
            return_tensors="pt"
        ),
        "response": relevance_labels,
        "reward": [rewards["relevance"] * rl_factors["relevance"]]
    }
)
for input_dict in inputs:
    trainer = trainers[input_dict["name"]]
    if input_dict["name"] == "qa":
        loss = trainer.update(
            inputs=input_dict["query"],
            responses=input_dict["response"],
            rewards=input_dict["reward"]
        )
    else:
        loss = trainer.update(
            inputs=input_dict["query"],
            responses=input_dict["response"],
            rewards=input_dict["reward"]
        )

for model_name in model_paths:
    trainers[model_name].model.save_pretrained(model_paths[model_name])
    trainers[model_name].tokenizer.save_pretrained(model_paths[model_name])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyError: 'input_ids'